###### * Base Source: https://www.kaggle.com/wangsg/a-self-attentive-model-for-knowledge-tracing
* My First Work: https://www.kaggle.com/leadbest/sakt-self-attentive-knowledge-tracing-submitter

1. Version 1: State Updates -> LB 0.765
2. Version 3: Random Selection of User Interactions -> LB 0.768
3. Version 6: Small Optimization -> LB 0.771?

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


In [ ]:
import gc
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import datatable as dt

data_types_dict = {
    'row_id':'int64',
 #   'timestamp': 'int64',
    'user_id': 'int32', 
    'content_id': 'int16', 
    'content_type_id':'int8', 
    #'task_container_id': 'int16',
    #'user_answer': 'int8',
    'answered_correctly': 'int8', 
    #'prior_question_elapsed_time': 'float32', 
    #'prior_question_had_explanation': 'bool'
}
target = 'answered_correctly'
data_df = dt.fread('../input/riiid-test-answer-prediction/train.csv', columns=set(data_types_dict.keys())).to_pandas()


In [ ]:
INPUT = '../../input/riiid-test-answer-prediction'

In [ ]:
#HDKIM
MAX_SEQ = 100
#HDKIMHDKIM
seq_length =100

## Load data

skills = data_df["content_id"].unique()
n_skill = len(skills)+1
print("number skills", len(skills))


del data_df
gc.collect()

In [ ]:
import pandas as pd
import numpy as np
from numba import jit

## Preprocess

In [ ]:
#HDKIM
import random
random.seed(1)
#HDKIMHDKIM

In [ ]:
MAX_SEQ

## Define model

In [ ]:

class SAINTPLUSModel(nn.Module):
    def __init__(self, n_skill,n_correctness, max_seq=MAX_SEQ, num_heads=8, embed_dim=128): #HDKIM 100->MAX_SEQ
        super(SAINTPLUSModel, self).__init__()
        
        
        self.n_skill = n_skill
        self.n_part = 9
        self.n_et = 302
        self.n_pos=101
        self.n_correctness=4
        
        
        self.embed_dim = embed_dim

        self.num_heads = num_heads

        self.correctness_embedding = nn.Embedding(self.n_correctness, embed_dim)
       
        self.pos_embedding = nn.Embedding(self.n_pos, embed_dim)
        
        self.ex_embedding = nn.Embedding(self.n_skill, embed_dim)
        
        self.part_embedding = nn.Embedding(self.n_part, embed_dim)
        

#         self.SAKT1 = SAKTmodule(embed_dim=embed_dim)
#         self.SAKT2 = SAKTmodule(embed_dim=embed_dim)
#         self.SAKT3 = SAKTmodule(embed_dim=embed_dim)

        self.encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=embed_dim,nhead=self.num_heads),num_layers=4)
        
        self.decoder = nn.TransformerDecoder(nn.TransformerDecoderLayer(d_model=embed_dim,nhead=self.num_heads),num_layers=4)
        
        #self.dropout = nn.Dropout(0.2)
        
       # self.elapT_weight = nn.Linear(1,embed_dim)
        
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self,excercise,correctness, parts, src_pad_mask , tgt_pad_mask,src_mask, tgt_mask ,pos_id_e ,pos_id_d):
        
        
    
        
     #  # pos_id_e = (torch.arange(x.size(1)) - torch.array(cum_id) ).unsqueeze(0).to(device)
        
    #  #  pos_id_d = (torch.arange(x.size(1)) - torch.array([0]+cum_id[:-1]) ).unsqueeze(0).to(device)

        
        
     #   lag_embedding
        
      #  interaction_embedding
        
      #  self.lag_embeding()
     #   self.elapsed_embedding()
        
       # print(elap_t.float().unsqueeze(- 1).shape)
        #print(pos_id_d.min())
        #print(pos_id_d.max())
        
        d = (self.correctness_embedding(correctness ) + self.pos_embedding(pos_id_d)).permute(1, 0, 2)
        
        e = (self.ex_embedding(excercise ) + self.pos_embedding(pos_id_e)+self.part_embedding(parts)).permute(1, 0, 2)

       
    
        #print(e.shape)
        
    #    e  = self.encoder(e,mask=src_mask.repeat(self.num_heads,1,1),src_key_padding_mask =src_pad_mask)
        
       # print(e)
        
    #    d = self.decoder(d,e,tgt_mask = tgt_mask.repeat(self.num_heads,1,1),memory_mask = src_mask.repeat(self.num_heads,1,1),tgt_key_padding_mask = tgt_pad_mask, memory_key_padding_mask = src_pad_mask  )

        
       # print(e)
        e  = self.encoder(e,mask=src_mask.repeat_interleave(self.num_heads,dim=0),src_key_padding_mask =src_pad_mask)
        
     #   print(e)
        
        d = self.decoder(d,e,tgt_mask = tgt_mask.repeat_interleave(self.num_heads,dim=0),memory_mask = src_mask.repeat_interleave(self.num_heads,dim=0),tgt_key_padding_mask = tgt_pad_mask, memory_key_padding_mask = src_pad_mask  )
    # train_loss: 0.20374205638760184, lr:0.00064275000000000    
    # valid_loss: 0.0952739624484875, auc: 0.9894335898260332 
     
        
        
        

        return self.pred(e).transpose(1,0).squeeze(-1) , src_pad_mask
    
    
    
    
    
    

    

In [ ]:
#torch.repeat_interleave()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = SAKTModel(n_skill, embed_dim=128)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.99, weight_decay=0.005)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#criterion = nn.BCEWithLogitsLoss()

#model.to(device)
#criterion.to(device)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    
    tbar = tqdm(dataloader)
    
    for item in tbar:
        optimizer.zero_grad()
        
        inp = item[0]
        label = item[1].to(device)#.float().to(device)
        
        for k, v in inp.items():
          
            inp[k] = v.to(device)
        
       # print(inp)
      
        output, msk = model( **inp)
        
        #print(output)
     #   print(output.dtype)
    
    
       # print(msk.shape)
      #  print(label.shape)
     #   print(output.shape)
        
     #   print(inp['src_pad_mask'][0])
     #   print(inp['src_mask'][0])
     #   print(inp['src_mask'][:,0])
     #   print(inp['tgt_pad_mask'][0])
     #   print(inp['correctness'][0])
        
        #print((label > ))
        
      #  print(label.dtype)
       # print(torch.any(label.flatten()[~msk.flatten()]<0))
        
       # print( (~msk).int().float())
    
      #  print(output.shape)
     #   print((output * (~msk).int().float()).shape)
     #   print((output * (~msk).int().float())[0])
        
        
      #  print(((label * (~msk).int()).float() >=0).all())
      
       # loss = loss_fn(output * (~msk).int().float() , (label * (~msk).int()).float() )
    
        print( ((label * (~msk).int()).float()>=0).all())
        
        print((label * (~msk).int()).float()[0])
        print(inp['correctness'][0])
        print(label[0][(~msk[0])])
        
        loss = loss_fn(output , (label * (~msk).int()).float() , weight = (~msk).int())
        
        print(loss)
        
        loss.backward()
        
        optimizer.step()
        
        if not  scheduler.__class__ ==  torch.optim.lr_scheduler.ReduceLROnPlateau:
            scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, scheduler, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    labels = []
    
    tbar = tqdm(dataloader)
    
    for item in tbar:
     
        inp = item[0]
        label = item[1].to(device)
        
        for k, v in inp.items():
          
            inp[k] = v.to(device)

        
      
        output, msk = model( **inp)
        
        loss = loss_fn(output , (label * (~msk).int()).float() , weight = (~msk).int())
        
        output = torch.sigmoid(output)
        
         
        output = output.flatten()[(~msk).flatten()]
        label = label.flatten()[(~msk).flatten()] 
        
        
        #labels.extend(label.view(-1).data.cpu().numpy())
        #valid_preds.extend(output.view(-1).data.cpu().numpy())
       
        labels.extend(label.data.cpu().numpy())
        valid_preds.extend(output.data.cpu().numpy())
        
        #valid_preds.append(output.sigmoid().detach().cpu().numpy())
       # labels.append(label.detach().cpu().numpy()[0])
        
        if scheduler.__class__ ==  torch.optim.lr_scheduler.ReduceLROnPlateau:
            scheduler.step(final_loss)
    
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    auc = roc_auc_score(labels, valid_preds)
    
    return final_loss,valid_preds,auc
    

def inference_fn(model, dataloader, device):
    model.eval()
    outs = []
    
    tbar = tqdm(dataloader)
    
    for item in tbar:
        optimizer.zero_grad()
        
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        label = item[2].to(device).float()

        with torch.no_grad():
            output, _ = model(x, target_id)
            
        output = torch.sigmoid(output)
        output = output[:, -1]
        
        outs.extend(output.data.cpu().numpy())
    
    return outs


def infer_features_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for item in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs,mode='get_features')
        
#         print(len(outputs))
        
#         for i in range(len(outputs)):
#             print(outputs[i].shape)
            
#         print(torch.cat(outputs,axis=1).shape)
        
        preds.append(torch.cat(outputs,axis=1).detach().cpu().numpy())
        
        
        
    preds = np.concatenate(preds)
    
    return preds
   
    

In [ ]:
def accuracy_score():
    
    pred = (torch.sigmoid(output) >= 0.5).long()

    num_corrects += (pred == label).sum().item()
    num_total += len(label)

    auc = roc_auc_score(labels, outs)
    acc = num_corrects / num_total
    


In [ ]:
#chequear q quc no sea solo en ultima pregunta

In [ ]:
class MaskedMSELoss(torch.nn.Module):
    def __init__(self):
        super(MaskedMSELoss, self).__init__()

    def forward(self, input, target, mask):
        diff2 = (torch.flatten(input) - torch.flatten(target)) ** 2.0
        sum2 = 0.0
        num = 0

        flat_mask = torch.flatten(mask)
        assert(len(flat_mask) == len(diff2))
        for i in range(len(diff2)):
            if flat_mask[i] == 1:
                sum2 += diff2[i]
                num += 1

        return sum2 / num

In [ ]:

EPOCHS = 2

LR = 6e-4

BATCH_SIZE = 128

EARLY_STOPPING_STEPS = 5

EARLY_STOP = False

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    #os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler



In [ ]:
class NoamLR(_LRScheduler):
    """
    Implements the Noam Learning rate schedule. This corresponds to increasing the learning rate
    linearly for the first ``warmup_steps`` training steps, and decreasing it thereafter proportionally
    to the inverse square root of the step number, scaled by the inverse square root of the
    dimensionality of the model. Time will tell if this is just madness or it's actually important.
    Parameters
    ----------
    warmup_steps: ``int``, required.
        The number of steps to linearly increase the learning rate.
    """
    def __init__(self, optimizer, warmup_steps):
        self.warmup_steps = warmup_steps
        super().__init__(optimizer)

    def get_lr(self):
        last_epoch = max(1, self.last_epoch)
        scale = self.warmup_steps ** 0.5 * min(last_epoch ** (-0.5), last_epoch * self.warmup_steps ** (-1.5))
        return [base_lr * scale for base_lr in self.base_lrs]


In [ ]:
    
def train_pipeline(train_dataset,valid_dataset, seed,fold,verbose=False,**kwargs):  
    
    seed_everything(seed)
    
   
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
   
    model = SAINTPLUSModel(n_skill,n_correctness=3, embed_dim=128)
    
    model.to(DEVICE)
    
    #initialize_from_past_model(model, f"../results/original_torch_moa_smoothed_lrplateau_5_folds_AUX_SEED{seed}_FOLD{fold}.pth")#,freeze_first_layer=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-6)
    #optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer, pct_start=0.1, div_factor=1e3, 
     #                                         max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    scheduler = NoamLR(optimizer,12000) #optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=3)
    
    
    
   # loss_val = nn.BCEWithLogitsLoss().to(DEVICE)
   # loss_tr =  nn.BCEWithLogitsLoss().to(DEVICE)
    
    loss_val = nn.functional.binary_cross_entropy_with_logits
    loss_tr =  nn.functional.binary_cross_entropy_with_logits
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    #todo el guardado de los resultados se puede mover a kfold que si tiene info de los indices
    #oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    best_auc = 0
    
    
    
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        if verbose:
            print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}, lr:{scheduler.get_last_lr()[0]}")
        valid_loss, valid_preds ,auc = valid_fn(model,scheduler, loss_val, validloader, DEVICE)
        if verbose:
            print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}, auc: {auc} ")

    #         if valid_loss < best_loss:

    #             best_loss = valid_loss
    #             oof = valid_preds
    #             torch.save(model.state_dict(), "SAKT-HDKIM.pt")

        if auc > best_auc:

            best_auc = auc
            oof = valid_preds
            torch.save(model.state_dict(), "SAKT-HDKIM.pt")



       #     torch.save(model.state_dict(), f"../results/{exp_name}_SEED{seed}_FOLD{fold}.pth")

        elif(EARLY_STOP == True):

            early_step += 1
            if (early_step >= early_stopping_steps):
                break

    
    #--------------------- PREDICTION---------------------
   
  #  testdataset = TestDataset(X_test)
  #  testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#          num_features= X_train.shape[1] ,
#         num_targets=  y_train.shape[1],
#         hidden_size=hidden_size,**kwargs
#     )
    
#     model.load_state_dict(torch.load(f"../results/FOLD{fold}_{exp_name}.pth"))
   # model.to(DEVICE)
    
    #predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    
    #predictions = inference_fn(model, testloader, DEVICE)
    
    return oof


In [ ]:
import gc


## Test

In [ ]:
import pickle as pk

In [ ]:
group = pd.read_pickle('../input/saintex1inference/saintEX1_groups_LAST_100_with_cum_ap.pkl')

In [ ]:
group.head()

In [ ]:
(group.apply(len)==3).all()

In [ ]:
#samples = .set_index('user_id')#groupby('user_id')

In [ ]:
class SAINTDataset(Dataset):
    def __init__(self, group, n_skill, max_seq=MAX_SEQ): #HDKIM 100
        super(SAINTDataset, self).__init__()
        self.max_seq = max_seq
        self.n_skill = n_skill
        self.samples = group
        
        self.dummy_correctness = 2
        self.dummy_elapsed_t = 0
        self.dummy_pos = 100
        
        self.forward_peak_mask = self._forward_peak_mask()

        #         self.user_ids = [x for x in group.index]
      # self.user_ids = []
      #  for user_id in group.index:
      #      q, qa = group[user_id]
      #      if len(q) < 2: #HDKIM 10
      #           continue
       #     self.user_ids.append(user_id)

        #HDKIM Memory reduction
        #if len(q)>self.max_seq:
        #    group[user_id] = (q[-self.max_seq:],qa[-self.max_seq:])
        
    def _forward_peak_mask(self):
        
        future_mask = np.triu(np.ones((self.max_seq, self.max_seq)), k=1).astype('bool')
        return torch.from_numpy(future_mask)

        
        
        
        
    def _get_tgt_msk_from_src_msk(self,tgt_msk):
       
        
        d_future_mask = torch.cat( (torch.ones((1,self.max_seq),dtype=torch.bool),torch.cat((torch.zeros((self.max_seq-1,1),dtype=torch.bool),tgt_msk[0:-1,0:-1]),dim=1)),dim=0)

        d_future_mask[0,0] = False
        
        return d_future_mask
   
    def _add_container_info_to_mask(self,res_ar,rep_list):
      
        for i,num in enumerate(rep_list):
    
            res_ar[i,i-num:i] = True

        return res_ar
        
        

    def __len__(self):
        return len(self.samples)
        #return len(self.samples)

    def __getitem__(self, index):
        #user_id = self.user_ids[index]
        cum_id_,q_, qa_ = self.samples.iloc[index]
        #q_, qa_ = self.samples.iloc[index]
        seq_len = len(q_)

        q = np.ones(self.max_seq, dtype=int) *-1
        qa = np.ones(self.max_seq, dtype=int)*-1
        cum_id = np.zeros(self.max_seq, dtype=int)
        
        if seq_len >= self.max_seq:
            #HDKIM
#             if random.random()>0.02:
#                 start = random.randint(0,(seq_len-self.max_seq))
#                 end = start + self.max_seq
#                 q[:] = q_[start:end]
#                 qa[:] = qa_[start:end]
#             else:
#                 #HDKIMHDKIM
            cum_id[:] = cum_id_[-self.max_seq:]
            q[:] = q_[-self.max_seq:]
            qa[:] = qa_[-self.max_seq:]
            
            
        else:
            
            #HDKIM
#             if random.random()>0.02:
#                 #HDKIMHDKIM
#                 start = 0
#                 end = random.randint(2,seq_len)
#                 seq_len = end - start
#                 q[-seq_len:] = q_[0:seq_len]
#                 qa[-seq_len:] = qa_[0:seq_len]
#             else:
                #HDKIMHDKIM
            cum_id[:seq_len] = cum_id_
            q[:seq_len] = q_
            qa[:seq_len] = qa_
            

        
        
        x = np.ones(self.max_seq, dtype=int)* -1
        
        x[1:seq_len] = qa[:seq_len-1].copy()
        
      #  if seq_len < self.max_seq:
            
         #   x[-seq_len] = self.dummy_correctness
       # else:
    
        x[0] = self.dummy_correctness
        
        
#         x = np.zeros(self.max_seq, dtype=int)
#         x[1:] = q[:-1].copy()
#         x[0] = self.dummy_position
        
        
        
#         x = np.zeros(self.max_seq, dtype=int)
#         x[1:] = q[:-1].copy()
#         x[0] = self.dummy_elapt
        
        
        
        
#         x = np.zeros(self.max_seq, dtype=int)
#         x[1:] = q[:-1].copy()
#         x[0] = self.dummy_lagt
        
        
        target_id = torch.from_numpy(q)

        label = torch.from_numpy(qa)

        correctness = torch.from_numpy(x)
        

        #x += (qa[:-1] == 1) * self.n_skill #this is special to sakt
        
        
        src_pad_mask =  target_id ==-1
        
        tgt_pad_mask = correctness ==-1
        
           
        
        src_mask =  self._add_container_info_to_mask(self.forward_peak_mask.clone(),cum_id)
        
        
        tgt_mask = self._get_tgt_msk_from_src_msk(src_mask)
        
        
        ###
        
        pos_id_e = torch.arange(target_id.size(0)) - torch.tensor(cum_id) 
        
       
              
        cum_id2= cum_id.copy()
        
        cum_id2[1:] = cum_id[:-1]
        
        cum_id2[0] = 0
        

       
        pos_id_d = torch.arange(target_id.size(0)) - torch.tensor(cum_id2 ) 

        
        
     #   lag_embedding
        
      #  interaction_embedding
        
        
    
        
        
     #   lag_embedding
        
      #  interaction_embedding
        
        inp = dict(excercise = target_id +1,correctness = correctness+1 ,
                   
                   src_mask = src_mask, tgt_mask=tgt_mask,
                   
                   src_pad_mask = src_pad_mask, tgt_pad_mask=tgt_pad_mask,
                   
                   pos_id_e = pos_id_e ,pos_id_d=pos_id_d )
      
    
        
        return  inp, label

In [ ]:
questions_df=pd.read_csv('../input/riiid-test-answer-prediction/questions.csv',index_col = 'question_id')

In [ ]:
questions_df.head()

In [ ]:
class TestDataset(Dataset):
    def __init__(self, samples, test_df, n_skill, max_seq=MAX_SEQ): #HDKIM 100
        super(TestDataset, self).__init__()
        self.samples = samples
        self.users = [x for x in test_df.index.unique()]
        self.test_df = test_df
        
        self.n_skill = n_skill
        self.max_seq = max_seq
        
        self.dummy_correctness = 2
        self.dummy_elapsed_t = 0
        self.dummy_pos = 100
        
        self.forward_peak_mask = self._forward_peak_mask()
        self.questions_df = questions_df
        
    
    def _forward_peak_mask(self):
        
        future_mask = np.triu(np.ones((self.max_seq, self.max_seq)), k=1).astype('bool')
        return torch.from_numpy(future_mask)

        
        
        
        
    def _get_tgt_msk_from_src_msk(self,tgt_msk):
       
        
        d_future_mask = torch.cat( (torch.ones((1,self.max_seq),dtype=torch.bool),torch.cat((torch.zeros((self.max_seq-1,1),dtype=torch.bool),tgt_msk[0:-1,0:-1]),dim=1)),dim=0)

        d_future_mask[0,0] = False
        
        return d_future_mask
   
    def _add_container_info_to_mask(self,res_ar,rep_list):
      
        for i,num in enumerate(rep_list):
    
            res_ar[i,i-num:i] = True

        return res_ar

    def __len__(self):
        return len(self.users)

    def __getitem__(self, index):
        
        user_id = self.users[index]
        
        test_info = self.test_df.loc[user_id]
       
        q = np.ones(self.max_seq, dtype=int) *-1
        qp = np.ones(self.max_seq, dtype=int)*-1
      #  qet = np.ones(self.max_seq, dtype=int)*-1
        qa = np.ones(self.max_seq, dtype=int)*-1
        cum_id = np.zeros(self.max_seq, dtype=int)
        
        x = np.ones(self.max_seq, dtype=int)* -1
        
        q_t_ = np.array(test_info['content_id'],ndmin=1)
        
        
        
        qp_t_= np.array(test_info['part'],ndmin=1)
        
        
        
        
       # print(q_t_)
        test_seq_len = len(q_t_)
        
        qa_t_ = np.ones(test_seq_len, dtype=int)* -1
        #qet_t_ = np.ones(test_seq_len, dtype=int)* -1
        
        
        cum_id_t_ = np.arange(test_seq_len)
        
        
        
        if user_id in self.samples.index:
            
            cur_samp = self.samples[user_id]
            
            cum_id_,q_, qa_,qp_= cur_samp[:4]
            
            
            q_ = np.concatenate((q_,q_t_))
            qp_ = np.concatenate((qp_, qp_t_))
            
            cum_id_ = np.concatenate((cum_id_,cum_id_t_))
                        
            qa_ = np.concatenate((qa_,qa_t_))
         #   qet_ = np.concatenate((qet_,qet_t_))
            
            
        
        else:
            
            q_ = q_t_
            qp_ =qp_t_
            cum_id_ = cum_id_t_
            qa_ = qa_t_ #np.array(-1,ndmin=1)
        #    qet_ = qet_t_
            
        seq_len = len(q_)
        
        if seq_len >= self.max_seq:

            cum_id[:] = cum_id_[-self.max_seq:]
            q[:] = q_[-self.max_seq:]
            qp[:] = qp_[-self.max_seq:]
            qa[:] = qa_[-self.max_seq:]
            #qet[:] = qet_[-self.max_seq:]
            
            
          #  qet[1:]=qet[:-1]
            x[1:]=qa[:-1].copy()
            
            
            
            pred_rng =(self.max_seq-test_seq_len,self.max_seq)
            
        else:

            cum_id[:seq_len] = cum_id_
            q[:seq_len] = q_
            qp[:seq_len] = qp_
          #  qet[:seq_len] = qet_
            
            qa[:seq_len] = qa_           
            
           # qet[1:seq_len+1] = qet[:seq_len]
            x[1:seq_len+1] = qa[:seq_len].copy()
        
            pred_rng =(seq_len-test_seq_len,seq_len)
       
        
        
        #x[1:seq_len] = qa[:seq_len].copy()
        
      #  if seq_len < self.max_seq:
            
         #   x[-seq_len] = self.dummy_correctness
       # else:
    
        x[0] = self.dummy_correctness
        
        
        
       # qet = np.roll(qet,1)
       # qet[0] = self.dummy_elapsed_t
        
        
        
#         x = np.zeros(self.max_seq, dtype=int)
#         x[1:] = q[:-1].copy()
#         x[0] = self.dummy_position
        
        
        
#         x = np.zeros(self.max_seq, dtype=int)
#         x[1:] = q[:-1].copy()
#         x[0] = self.dummy_elapt
        
        
        
        
#         x = np.zeros(self.max_seq, dtype=int)
#         x[1:] = q[:-1].copy()
#         x[0] = self.dummy_lagt
        
        
        target_id = torch.from_numpy(q)

        label = torch.from_numpy(qa)
        
        parts = torch.from_numpy(qp)

        correctness = torch.from_numpy(x)
       # elap_t = torch.from_numpy(np.where(qet<-1,-1,qet))

        #x += (qa[:-1] == 1) * self.n_skill #this is special to sakt
        
        
        src_pad_mask =  target_id ==-1
        
        tgt_pad_mask = correctness ==-1
        
           
        
        src_mask =  self._add_container_info_to_mask(self.forward_peak_mask.clone(),cum_id)
        
        
        tgt_mask = self._get_tgt_msk_from_src_msk(src_mask)
        
        
        ###
        if cum_id[0]>0:
            
            cur_cum_ap = cum_id[0] 
            ctr=0
            
            while cur_cum_ap >0 :
                
                cum_id[ctr] = ctr
                
                ctr +=1
                
                cur_cum_ap = cum_id[ctr]
                
        
        pos_id_e = torch.arange(target_id.size(0)) - torch.tensor(cum_id) 
        
       
              
        cum_id2= cum_id.copy()
        
        cum_id2[1:] = cum_id[:-1]
        
        cum_id2[0] = - 100
        

        

       
        pos_id_d = torch.arange(target_id.size(0)) - torch.tensor(cum_id2 ) 

        
        
     #   lag_embedding
        
      #  interaction_embedding
        
        
    
        
        
     #   lag_embedding
        
      #  interaction_embedding
        
         
        inp = dict(excercise = target_id +1,correctness = correctness+1 ,parts =parts +1 ,# elap_t =elap_t+1 ,
                   
                   src_mask = src_mask, tgt_mask=tgt_mask,
                   
                   src_pad_mask = src_pad_mask, tgt_pad_mask=tgt_pad_mask,
                   
                   pos_id_e = pos_id_e  ,pos_id_d=pos_id_d )
      
    
        
        return  inp, pred_rng

pred_rng

np.array(prev_test_df.iloc[0]['content_id'])

test_df, sample_prediction_df =next(iter_test)




for (test_df, sample_prediction_df) in tqdm(iter_test):
    #HDKIM

    if (prev_test_df is not None) & (psutil.virtual_memory().percent<90):

        print(psutil.virtual_memory().percent)

        prev_test_df['answered_correctly'] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prev_test_df = prev_test_df[prev_test_df.content_type_id == False]

        #supposing we recieve task container_id alltogether
        prev_group = prev_test_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: ( np.arange(len(r)),
            r['content_id'].values,
            r['answered_correctly'].values))

        for prev_user_id in prev_group.index:

            prev_group_cum_ap = prev_group[prev_user_id][0]
            prev_group_content = prev_group[prev_user_id][1]
            prev_group_ac = prev_group[prev_user_id][2]
            if prev_user_id in group.index:
                group[prev_user_id] = (np.append(group[prev_user_id][0],prev_group_cum_ap),np.append(group[prev_user_id][1],prev_group_content), 
                                       np.append(group[prev_user_id][2],prev_group_ac))

            else:

                group[prev_user_id] = (prev_group_ac,prev_group_content,prev_group_ac)

            if len(group[prev_user_id][0])>MAX_SEQ:

                new_group_cum_ap = group[prev_user_id][0][-MAX_SEQ:]
                new_group_content = group[prev_user_id][1][-MAX_SEQ:]
                new_group_ac = group[prev_user_id][2][-MAX_SEQ:]

                group[prev_user_id] = (new_group_cum_ap,new_group_content,new_group_ac)

    prev_test_df = test_df.copy()

    #HDKIMHDKIM

    test_df = test_df[test_df.content_type_id == False]

    test_dataset = TestDataset(group, test_df.loc[test_df['content_type_id'] == 0].set_index('user_id'), n_skill=13524)
    test_dataloader = DataLoader(test_dataset, batch_size=12000, shuffle=False)

    # outs = []

    #for item in tqdm(test_dataloader):
    model.eval()
    final_loss = 0
    valid_preds = []
    # labels = []

    tbar = tqdm(test_dataloader)

    for item in tbar:

        inp = item[0]
        pred_rng = item[1]#.to(device)

        for k, v in inp.items():

            inp[k] = v.to(device)

        with torch.no_grad():
            output, msk = model( **inp)

        output = torch.sigmoid(output)
    # output = output[:, -1]

    # pred = (output >= 0.5).long()
    # loss = criterion(output, label)

    # val_loss.append(loss.item())
    # num_corrects += (pred == label).sum().item()
    # num_total += len(label)

    # labels.extend(label.squeeze(-1).data.cpu().numpy())
        for i in range(len(pred_rng[0])):

            valid_preds.extend(output[i,pred_rng[0][i]:pred_rng[1][i]].data.cpu().numpy())

    test_df.loc[test_df['content_type_id'] == 0,'answered_correctly'] =  valid_preds

    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

In [ ]:


def valid_last_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    labels = []
    
    tbar = tqdm(dataloader)
    
    for item in tbar:
     
        inp = item[0]
        label = item[1].to(device)
        
        for k, v in inp.items():
          
            inp[k] = v.to(device)

        
      
        output, msk = model( **inp)
        
        loss = loss_fn(output , (label * (~msk).int()).float() , weight = (~msk).int())
        
        output = torch.sigmoid(output)
        
        amin = (label==-1).int().argmax(1)-1

        lasts_idx = torch.where(amin != -1 , amin, MAX_SEQ-1)
       # 
        #[label[i,idx] for i,idx in enumerate(lasts_idx)]        
        
         
        output = torch.tensor([output[i,idx] for i,idx in enumerate(lasts_idx)])   #.flatten()[(~msk).flatten()]
        
        #print(output)
        
        label = torch.tensor([label[i,idx] for i,idx in enumerate(lasts_idx)])    #.flatten()[(~msk).flatten()] 
        #print(label)
        
        #break
        #labels.extend(label.view(-1).data.cpu().numpy())
        #valid_preds.extend(output.view(-1).data.cpu().numpy())
       
        labels.extend(label.data.cpu().numpy())
        valid_preds.extend(output.data.cpu().numpy())
        
        #valid_preds.append(output.sigmoid().detach().cpu().numpy())
       # labels.append(label.detach().cpu().numpy()[0])
        
#         if scheduler.__class__ ==  torch.optim.lr_scheduler.ReduceLROnPlateau:
#             scheduler.step(final_loss)
    
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    auc = roc_auc_score(labels, valid_preds)
    
    return final_loss,valid_preds,auc

In [ ]:


def inference_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
   # labels = []
    
    tbar = tqdm(dataloader)
    
    for item in tbar:
     
        inp = item[0]
        pred_rng = item[1]#.to(device)
        
        for k, v in inp.items():
          
            inp[k] = v.to(device)

        
      
        output, msk = model( **inp)
        
        #loss = loss_fn(output , (label * (~msk).int()).float() , weight = (~msk).int())
        
        output = torch.sigmoid(output)
        
       # amin = (label==-1).int().argmax(1)-1

       # lasts_idx = torch.where(amin != -1 , amin, MAX_SEQ-1)
       # 
        #[label[i,idx] for i,idx in enumerate(lasts_idx)]        
        
         
       # output = torch.tensor([output[i,idx[0]:idx[1]] for i,idx in enumerate(pred_rng)])   #.flatten()[(~msk).flatten()]
        #print(output)
        
        #label = torch.tensor([label[i,idx] for i,idx in enumerate(lasts_idx)])    #.flatten()[(~msk).flatten()] 
        #print(label)
        
        #break
        #labels.extend(label.view(-1).data.cpu().numpy())
        #valid_preds.extend(output.view(-1).data.cpu().numpy())
       
      #  labels.extend(label.data.cpu().numpy())
        for i,idx in enumerate(pred_rng):
            
            valid_preds.extend(output[i,idx[0]:idx[1]].data.cpu().numpy())
        
        #valid_preds.append(output.sigmoid().detach().cpu().numpy())
       # labels.append(label.detach().cpu().numpy()[0])
        
#         if scheduler.__class__ ==  torch.optim.lr_scheduler.ReduceLROnPlateau:
#             scheduler.step(final_loss)
    
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    auc = roc_auc_score(labels, valid_preds)
    
    return final_loss,valid_preds,auc

In [ ]:
model = SAINTPLUSModel(n_skill=13524,n_correctness=3, embed_dim=256)

model.load_state_dict(torch.load(f"../input/saint-4l-8h-emb256-extra-part-virtual/SAint_4l_8h_emb256_extra_part_virtual2.pt",map_location=device))
model.to(device)

In [ ]:
import psutil



test_df, sample_prediction_df =next(iter_test)

In [ ]:
prev_test_df = None

target_df = pd.read_pickle('../input/riiid-cross-validation-files/cv1_train.pickle')

# I use the middle 2.5M range here.
target_df = target_df[50_000_000:52_500_000]

class Iter_Valid(object):
    def __init__(self, df, max_user=1000):
        df = df.reset_index(drop=True)
        self.df = df
        self.user_answer = df['user_answer'].astype(str).values
        self.answered_correctly = df['answered_correctly'].astype(str).values
        df['prior_group_responses'] = "[]"
        df['prior_group_answers_correct'] = "[]"
        self.sample_df = df[df['content_type_id'] == 0][['row_id']]
        self.sample_df['answered_correctly'] = 0
        self.len = len(df)
        self.user_id = df.user_id.values
        self.task_container_id = df.task_container_id.values
        self.content_type_id = df.content_type_id.values
        self.max_user = max_user
        self.current = 0
        self.pre_user_answer_list = []
        self.pre_answered_correctly_list = []

    def __iter__(self):
        return self
    
    def fix_df(self, user_answer_list, answered_correctly_list, pre_start):
        df= self.df[pre_start:self.current].copy()
        sample_df = self.sample_df[pre_start:self.current].copy()
        df.loc[pre_start,'prior_group_responses'] = '[' + ",".join(self.pre_user_answer_list) + ']'
        df.loc[pre_start,'prior_group_answers_correct'] = '[' + ",".join(self.pre_answered_correctly_list) + ']'
        self.pre_user_answer_list = user_answer_list
        self.pre_answered_correctly_list = answered_correctly_list
        return df, sample_df

    def __next__(self):
        added_user = set()
        pre_start = self.current
        pre_added_user = -1
        pre_task_container_id = -1

        user_answer_list = []
        answered_correctly_list = []
        while self.current < self.len:
            crr_user_id = self.user_id[self.current]
            crr_task_container_id = self.task_container_id[self.current]
            crr_content_type_id = self.content_type_id[self.current]
            if crr_content_type_id == 1:
                # no more than one task_container_id of "questions" from any single user
                # so we only care for content_type_id == 0 to break loop
                user_answer_list.append(self.user_answer[self.current])
                answered_correctly_list.append(self.answered_correctly[self.current])
                self.current += 1
                continue
            if crr_user_id in added_user and ((crr_user_id != pre_added_user) or (crr_task_container_id != pre_task_container_id)):
                # known user(not prev user or differnt task container)
                return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
            if len(added_user) == self.max_user:
                if  crr_user_id == pre_added_user and crr_task_container_id == pre_task_container_id:
                    user_answer_list.append(self.user_answer[self.current])
                    answered_correctly_list.append(self.answered_correctly[self.current])
                    self.current += 1
                    continue
                else:
                    return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
            added_user.add(crr_user_id)
            pre_added_user = crr_user_id
            pre_task_container_id = crr_task_container_id
            user_answer_list.append(self.user_answer[self.current])
            answered_correctly_list.append(self.answered_correctly[self.current])
            self.current += 1
        if pre_start < self.current:
            return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
        else:
            raise StopIteration()

validaten_flg = True
if validaten_flg:
    iter_test = Iter_Valid(target_df,max_user=1000)
    predicted = []
    def set_predict(df):
        predicted.append(df)
else:
    import riiideducation
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict


for (test_df, sample_prediction_df) in tqdm(iter_test):
    #HDKIM

    if (prev_test_df is not None) & (psutil.virtual_memory().percent<90):

       # print(psutil.virtual_memory().percent)
        ans_cor = eval(test_df['prior_group_answers_correct'].iloc[0])
        
        prev_test_df['answered_correctly'] = [it for it in ans_cor if it>-1]
        #prev_test_df['elapsed_time'] = test_df.iloc[1:['prior_question_elapsed_time']
                                                    
        #prev_test_df = prev_test_df[prev_test_df.content_type_id == False]

        #supposing we recieve task container_id alltogether
        prev_group = prev_test_df[['user_id', 'content_id', 'answered_correctly','part']].groupby('user_id').apply(lambda r: ( np.arange(len(r)),
            r['content_id'].values,
            r['answered_correctly'].values,r['part'].values))

        for prev_user_id in prev_group.index:

            prev_group_cum_ap = prev_group[prev_user_id][0]
            prev_group_content = prev_group[prev_user_id][1]
            prev_group_ac = prev_group[prev_user_id][2]
            prev_group_part = prev_group[prev_user_id][3]
           # prev_group_elapt = np.array([np.nan]*len(prev_group_ac),ndmin=1)
            
            if prev_user_id in group.index:
                group[prev_user_id] = (np.append(group[prev_user_id][0],prev_group_cum_ap),np.append(group[prev_user_id][1],prev_group_content), 
                                       np.append(group[prev_user_id][2],prev_group_ac),np.append(group[prev_user_id][3],prev_group_part),group[prev_user_id][4])
                                       #,np.append(group[prev_user_id][4])#,prev_group_elapt))

            else:

                group[prev_user_id] = (prev_group_ac,prev_group_content,prev_group_ac,prev_group_part,prev_group_elapt)

            if len(group[prev_user_id][0])>MAX_SEQ:

                new_group_cum_ap = group[prev_user_id][0][-MAX_SEQ:]
                new_group_content = group[prev_user_id][1][-MAX_SEQ:]
                new_group_ac = group[prev_user_id][2][-MAX_SEQ:]
                new_group_part = group[prev_user_id][3][-MAX_SEQ:]
                new_group_elapt = group[prev_user_id][4][-MAX_SEQ:]

                group[prev_user_id] = (new_group_cum_ap,new_group_content,new_group_ac,new_group_part,new_group_elapt)
                
                
#     for cur_user_id,elap_t in test_df.iloc[1:].drop_duplicates('user_id')[['user_id','prior_question_elapsed_time']].values:
        
#         ar = group[cur_user_id][4].copy()
        
#         indx = -1
        
#         while (ar[indx]<0) or np.isnan(ar[indx]): 
                
#             indx-=1
        
        
#         ar = ar[:len(ar)+indx+1]
        
        
#         cur_group_cum_ap = group[cur_user_id][0][-MAX_SEQ:]
#         cur_group_content = group[cur_user_id][1][-MAX_SEQ:]
#         cur_group_ac = group[cur_user_id][2][-MAX_SEQ:]
#         cur_group_part = group[cur_user_id][3][-MAX_SEQ:]
                
        
#         group[cur_user_id] = (cur_group_cum_ap,cur_group_content,cur_group_ac, cur_group_part,np.append(ar,np.array([elap_t//1000]*(len(cur_group_cum_ap)-len(ar)) ))[-MAX_SEQ:])
        
    

    #HDKIMHDKIM

    test_df = test_df.loc[test_df.content_type_id == False]
    
    
    test_df['part'] = test_df['content_id'].map(questions_df['part'])
    
    prev_test_df = test_df.copy()
    
   
    
    test_dataset =   TestDataset(group, test_df.set_index('user_id'), n_skill=13524)                #TestDataset(group, test_df.loc[test_df['content_type_id'] == 0].set_index('user_id'), n_skill=13524)
    
    test_dataloader = DataLoader(test_dataset, batch_size=1200, shuffle=False)

    # outs = []

    #for item in tqdm(test_dataloader):
    model.eval()
    final_loss = 0
    valid_preds = []
    # labels = []

    tbar = tqdm(test_dataloader)

    for item in tbar:

        inp = item[0]
        pred_rng = item[1]#.to(device)

        for k, v in inp.items():

            inp[k] = v.to(device)

        with torch.no_grad():
            output, msk = model( **inp)

        output = torch.sigmoid(output)
    # output = output[:, -1]

    # pred = (output >= 0.5).long()
    # loss = criterion(output, label)

    # val_loss.append(loss.item())
    # num_corrects += (pred == label).sum().item()
    # num_total += len(label)

    # labels.extend(label.squeeze(-1).data.cpu().numpy())
        for i in range(len(pred_rng[0])):

            valid_preds.extend(output[i,pred_rng[0][i]:pred_rng[1][i]].data.cpu().numpy())

    test_df['answered_correctly'] =  valid_preds               #.loc[test_df['content_type_id'] == 0,'answered_correctly'] =  valid_preds
    
   # print(test_df[['row_id', 'answered_correctly']])
    print(test_df.loc[:,['row_id', 'answered_correctly']])
   # env.predict(test_df[['row_id', 'answered_correctly']])                 # .loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
    set_predict(test_df.loc[:,['row_id', 'answered_correctly']])
    #pbar.update(len(current_test))
    #env.predict(test_df[['row_id', 'answered_correctly']])                 # .loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

In [ ]:
import riiideducation

env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:


for (test_df, sample_prediction_df) in tqdm(iter_test):
    #HDKIM

    if (prev_test_df is not None) & (psutil.virtual_memory().percent<90):

       # print(psutil.virtual_memory().percent)
        ans_cor = eval(test_df['prior_group_answers_correct'].iloc[0])
        
        prev_test_df['answered_correctly'] = [it for it in ans_cor if it>-1]
        #prev_test_df['elapsed_time'] = test_df.iloc[1:['prior_question_elapsed_time']
                                                    
        #prev_test_df = prev_test_df[prev_test_df.content_type_id == False]

        #supposing we recieve task container_id alltogether
        prev_group = prev_test_df[['user_id', 'content_id', 'answered_correctly','part']].groupby('user_id').apply(lambda r: ( np.arange(len(r)),
            r['content_id'].values,
            r['answered_correctly'].values,r['part'].values))

        for prev_user_id in prev_group.index:

            prev_group_cum_ap = prev_group[prev_user_id][0]
            prev_group_content = prev_group[prev_user_id][1]
            prev_group_ac = prev_group[prev_user_id][2]
            prev_group_part = prev_group[prev_user_id][3]
           # prev_group_elapt = np.array([np.nan]*len(prev_group_ac),ndmin=1)
            
            if prev_user_id in group.index:
                group[prev_user_id] = (np.append(group[prev_user_id][0],prev_group_cum_ap),np.append(group[prev_user_id][1],prev_group_content), 
                                       np.append(group[prev_user_id][2],prev_group_ac),np.append(group[prev_user_id][3],prev_group_part),group[prev_user_id][4])
                                       #,np.append(group[prev_user_id][4])#,prev_group_elapt))

            else:

                group[prev_user_id] = (prev_group_ac,prev_group_content,prev_group_ac,prev_group_part,np.array([np.nan],ndmin=1))

            if len(group[prev_user_id][0])>MAX_SEQ:

                new_group_cum_ap = group[prev_user_id][0][-MAX_SEQ:]
                new_group_content = group[prev_user_id][1][-MAX_SEQ:]
                new_group_ac = group[prev_user_id][2][-MAX_SEQ:]
                new_group_part = group[prev_user_id][3][-MAX_SEQ:]
                new_group_elapt = group[prev_user_id][4][-MAX_SEQ:]

                group[prev_user_id] = (new_group_cum_ap,new_group_content,new_group_ac,new_group_part,new_group_elapt)
                
                
#     for cur_user_id,elap_t in test_df.iloc[1:].drop_duplicates('user_id')[['user_id','prior_question_elapsed_time']].values:
        
#         ar = group[cur_user_id][4].copy()
        
#         indx = -1
        
#         while (ar[indx]<0) or np.isnan(ar[indx]): 
                
#             indx-=1
        
        
#         ar = ar[:len(ar)+indx+1]
        
        
#         cur_group_cum_ap = group[cur_user_id][0][-MAX_SEQ:]
#         cur_group_content = group[cur_user_id][1][-MAX_SEQ:]
#         cur_group_ac = group[cur_user_id][2][-MAX_SEQ:]
#         cur_group_part = group[cur_user_id][3][-MAX_SEQ:]
                
        
#         group[cur_user_id] = (cur_group_cum_ap,cur_group_content,cur_group_ac, cur_group_part,np.append(ar,np.array([elap_t//1000]*(len(cur_group_cum_ap)-len(ar)) ))[-MAX_SEQ:])
        
    

    #HDKIMHDKIM

    test_df = test_df.loc[test_df.content_type_id == False]
    
    
    test_df['part'] = test_df['content_id'].map(questions_df['part'])
    
    prev_test_df = test_df.copy()
    
   
    
    test_dataset =   TestDataset(group, test_df.set_index('user_id'), n_skill=13524)                #TestDataset(group, test_df.loc[test_df['content_type_id'] == 0].set_index('user_id'), n_skill=13524)
    
    test_dataloader = DataLoader(test_dataset, batch_size=1200, shuffle=False)

    # outs = []

    #for item in tqdm(test_dataloader):
    model.eval()
    final_loss = 0
    valid_preds = []
    # labels = []

    tbar = tqdm(test_dataloader)

    for item in tbar:

        inp = item[0]
        pred_rng = item[1]#.to(device)

        for k, v in inp.items():

            inp[k] = v.to(device)

        with torch.no_grad():
            output, msk = model( **inp)

        output = torch.sigmoid(output)
    # output = output[:, -1]

    # pred = (output >= 0.5).long()
    # loss = criterion(output, label)

    # val_loss.append(loss.item())
    # num_corrects += (pred == label).sum().item()
    # num_total += len(label)

    # labels.extend(label.squeeze(-1).data.cpu().numpy())
        for i in range(len(pred_rng[0])):

            valid_preds.extend(output[i,pred_rng[0][i]:pred_rng[1][i]].data.cpu().numpy())

    test_df['answered_correctly'] =  valid_preds               #.loc[test_df['content_type_id'] == 0,'answered_correctly'] =  valid_preds
               #.loc[test_df['content_type_id'] == 0,'answered_correctly'] =  valid_preds
    
   # print(test_df[['row_id', 'answered_correctly']])
    
   # env.predict(test_df[['row_id', 'answered_correctly']])
    
    env.predict(test_df.loc[:,['row_id', 'answered_correctly']])                 # .loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

sub = pd.read_csv('./submission.csv')

sub.info()




#for (test_df, sample_prediction_df) in tqdm(iter_test):
    #HDKIM

#HDKIMHDKIM

test_df = test_df.loc[test_df.content_type_id == False]

test_df['part']=questions_df.loc[test_df['content_id'],'part']

test_dataset =   TestDataset(group, test_df.set_index('user_id'), n_skill=13524)                #TestDataset(group, test_df.loc[test_df['content_type_id'] == 0].set_index('user_id'), n_skill=13524)

test_dataloader = DataLoader(test_dataset, batch_size=1200, shuffle=False)

# outs = []

#for item in tqdm(test_dataloader):
model.eval()
final_loss = 0
valid_preds = []
# labels = []

tbar = tqdm(test_dataloader)

for item in tbar:

    inp = item[0]
    pred_rng = item[1]#.to(device)

    for k, v in inp.items():

        inp[k] = v.to(device)

    with torch.no_grad():
        output, msk = model( **inp)

    output = torch.sigmoid(output)
# output = output[:, -1]

# pred = (output >= 0.5).long()
# loss = criterion(output, label)

# val_loss.append(loss.item())
# num_corrects += (pred == label).sum().item()
# num_total += len(label)

# labels.extend(label.squeeze(-1).data.cpu().numpy())
    for i in range(len(pred_rng[0])):

        valid_preds.extend(output[i,pred_rng[0][i]:pred_rng[1][i]].data.cpu().numpy())

test_df['answered_correctly'] =  valid_preds               #.loc[test_df['content_type_id'] == 0,'answered_correctly'] =  valid_preds

env.predict(test_df[['row_id', 'answered_correctly']])                 # .loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])